<a href="https://colab.research.google.com/github/pallav0007/portfolio_management/blob/main/Porfolio_manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Installing required Dependencies

In [ ]:
!pip install tensorflow-gpu==1.15.0 tensorflow==1.15.0 stable-baselines gym-anytrading gym # for creating environment and training model
!pip install nsepy # for getting real time NSE data
!pip install ta  # for technical indicator analysis

###Importing the libraries

In [ ]:
import gym
import gym_anytrading

# Stable baselines - rl stuff
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import A2C

# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from nsepy import get_history
import datetime
import ta
from ta.momentum import RSIIndicator
from ta.trend import MACD
import pandas as pd
import pandas
import random
import datetime
import pandas as pd

###Function to create custom Dataframe with custom indicators


In [ ]:
dt=datetime.date.today()
def in_squeeze(df):
    return (df['lower_band'] > df['lower_keltner']) and (df['upper_band'] < df['upper_keltner'])
def score_col(dat):
    score=0
    if (dat["rsi"]) >= 60:
        score+=5
    if dat['squeeze_on']:
        score+=10
    if dat["dev_per"] > 40:
        score+=5
    if dat["macd"]-dat["mach_Signal"]>=0:
        score+=5
    return score
def get_dataframe(stock,index=False):
    df = get_history(symbol=stock,
                      start=dt - datetime.timedelta(2000),
                     index=index,
                      end=dt)
    df['20sma'] = df['Close'].rolling(window=20).mean()
    df['stddev'] = df['Close'].rolling(window=20).std()
    df['lower_band'] = df['20sma'] - (2 * df['stddev'])
    df['upper_band'] = df['20sma'] + (2 * df['stddev'])
    df["cp"]=df["Close"].shift(1)
    df["p1"]=abs(df["High"]-df["cp"])
    df["p2"]=abs(df["Low"]-df["cp"])
    df['p3'] = abs(df['High'] - df['Low'])
    df["TR"]=df[["p1","p2","p3"]].max(axis=1)
    df['ATR'] = df['TR'].rolling(window=20).mean()
    df["avgvol3"] = df["Volume"].rolling(window=10).mean()
    df["avgvol10"] = df["Volume"].rolling(window=3).mean()
    df['lower_keltner'] = df['20sma'] - (df['ATR'] * 1.5)
    df['upper_keltner'] = df['20sma'] + (df['ATR'] * 1.5)
    rsi_indicator = RSIIndicator(df["Close"], 15)
    df['rsi'] = rsi_indicator.rsi()
    macd_int = MACD(df["Close"], 26, 12, 9)
    df["macd"] = macd_int.macd()
    df['mach_Histogram'] = macd_int.macd_diff()
    df['mach_Signal'] = macd_int.macd_signal()

    df['squeeze_on'] = df.apply(in_squeeze, axis=1)
    df["dev_per"]=(df["Deliverable Volume"]/df["Volume"])*100

    df["Date"]=df.index
    df.set_index('Date', inplace=True)
    df["score"]=df.apply(score_col,axis=1)
    df = df.dropna(how='any')
    df=df.drop("Symbol",axis=1)
    df = df.drop("Series",axis=1)
    return df


###Calling the database using the functions above


In [ ]:
stock=input()
df=get_dataframe(stock.upper())
# df=pd.read_csv("/content/drive/MyDrive/colab_custom_data.csv") alternatively use preprocessed file

###Create Environment and Add custom signals

In [ ]:
env = gym.make('stocks-v0', df=df, frame_bound=(5,250), window_size=5)


In [ ]:
from gym_anytrading.envs import StocksEnv
def add_signals(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Low'].to_numpy()[start:end]
    signal_features = env.df.loc[:,['Prev Close','Low', 'Close', 'VWAP', 'Volume','Trades', '%Deliverble', '20sma',
       'stddev','ATR','avgvol10', 'rsi','macd', 'mach_Histogram', 'dev_per', 'score',
       'squeeze_on_True']].to_numpy()[start:end]
    return prices, signal_features
class MyCustomEnv(StocksEnv):
    _process_data = add_signals
    
env2 = MyCustomEnv(df=df, window_size=12, frame_bound=(12,50))
env_maker = lambda: env2
env = DummyVecEnv([env_maker])

###Choose a policy and Train Model


In [ ]:
model = A2C('MlpLstmPolicy', env, verbose=1) 
model.learn(total_timesteps=1000000)
# optional save model on drive to save training time
# model.save("/content/drive/MyDrive/ri_model")
# and load saved model with
# model = A2C.load("/content/drive/MyDrive/ri_model", verbose=1)

###Use Model to get Predictions


In [ ]:
env = MyCustomEnv(df=df, window_size=12, frame_bound=(80,250))
obs = env.reset()
while True: 
    obs = obs[np.newaxis, ...]
    x=model.predict(obs)
    action, _states = x
    print(x)
    obs, rewards, done, info = env.step(action)
    if done:
        print("info", info)
        break


###visualize the predictions

In [ ]:
plt.figure(figsize=(15,6))
plt.cla()
env.render_all()
plt.show()